In [20]:
import os
import sys
import json
import pandas as pd
import numpy as np

from azure_blob import download_blob_file
from utilities import string_to_dict

In [21]:
# Download du .csv originel depuis le serveur de stockage

filename = "metrics.csv"
path = "../data/"

download_blob_file(file_name=filename , local_path=path)

metrics.csv already in path ../data/.


In [22]:
# Création du Dataframe à partir du .csv importé
# Filtration du Df via la colonne "created_at"
# Suppression de toutes les colonnes vides

df= pd.read_csv("../data/metrics.csv").sort_values("created_at").dropna(axis=1, how="all",)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164430 entries, 307539 to 1164429
Data columns (total 9 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1164430 non-null  int64  
 1   status                     1164430 non-null  object 
 2   created_at                 1164430 non-null  object 
 3   machineId                  1164430 non-null  int64  
 4   connected_operators        1164430 non-null  object 
 5   varnishLevelsTargetvolume  1164430 non-null  float64
 6   varnishLevelsTotalvolume   1164430 non-null  int64  
 7   modules                    1164430 non-null  object 
 8   events                     1164430 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 88.8+ MB


,id,status,created_at,machineId,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
307539,4169748,WARNING,2022-04-15 05:55:06.678000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
307540,4169749,WARNING,2022-04-15 05:55:06.829000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...","[{""source"": ""PLC"", ""message"": "" JV-Ti non prêt..."
307537,4169753,WARNING,2022-04-15 05:55:14.494000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
307541,4169758,WARNING,2022-04-15 05:55:24.503000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
307542,4169763,WARNING,2022-04-15 05:55:34.515000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


In [23]:
# Modification du Dataframe pour ne garder que les colonnes qui nous intéresse

df = df[["id","status","created_at","events"]]
df.head()

,id,status,created_at,events
307539,4169748,WARNING,2022-04-15 05:55:06.678000,[]
307540,4169749,WARNING,2022-04-15 05:55:06.829000,"[{""source"": ""PLC"", ""message"": "" JV-Ti non prêt..."
307537,4169753,WARNING,2022-04-15 05:55:14.494000,[]
307541,4169758,WARNING,2022-04-15 05:55:24.503000,[]
307542,4169763,WARNING,2022-04-15 05:55:34.515000,[]


In [24]:
# Suppression de toutes les lignes sans données dans la colonne "events"
# Remise de l'index à zéro

df = df[df.events != "[]"].reset_index(drop=True)
df.head()

,id,status,created_at,events
0,4169749,WARNING,2022-04-15 05:55:06.829000,"[{""source"": ""PLC"", ""message"": "" JV-Ti non prêt..."
1,4170152,IDLE,2022-04-15 06:06:35.404000,"[{""source"": ""iFoil"", ""message"": "" JV-Ti non pr..."
2,4170167,IDLE,2022-04-15 06:07:05.443000,"[{""source"": ""PLC"", ""message"": "" Disponible"", ""..."
3,4170172,IDLE,2022-04-15 06:07:15.471000,"[{""source"": ""PLC"", ""message"": "" Chargeur: mode..."
4,4170207,IDLE,2022-04-15 06:08:25.616000,"[{""source"": ""PLC"", ""message"": "" Préchauffage"",..."


In [25]:
## Modification du Dataframe pour ne garder que les colonnes qui nous intéresse
## Suppression de toutes les lignes sans données dans la colonne "events"
## Remise de l'index à zéro

df = df[["id","status","created_at","events"]][df.events != "[]"].reset_index(drop=True)
# df.head()

In [26]:
# Création d'un .csv permettant de sauvegardé les modification faites

df.to_csv(path + "metrics_events.csv")

In [27]:
def convert_str_to_list(series):
    return pd.DataFrame(series.apply(lambda x :json.loads(x)), columns=[series.name])

In [28]:
df_events = convert_str_to_list(df.events)
df_events.head()

,events
0,"[{'source': 'PLC', 'message': ' JV-Ti non prêt..."
1,"[{'source': 'iFoil', 'message': ' JV-Ti non pr..."
2,"[{'source': 'PLC', 'message': ' Disponible', '..."
3,"[{'source': 'PLC', 'message': ' Chargeur: mode..."
4,"[{'source': 'PLC', 'message': ' Préchauffage',..."


In [29]:
events_keys = []
for i in range(0, len(df_events)) :
    if len(df_events.events.loc[i]) != 0 :
        event_keys = df_events.events.loc[i][0].keys()
        if event_keys not in events_keys :
            events_keys.append(event_keys)

In [30]:
events_dict = {}
for key in event_keys :
    events_dict[key] = None

In [31]:
df_events['Length'] = df_events.events.map(len)

In [32]:
df_events = pd.DataFrame(df_events.events.to_list(), dtype=object)

df_events.info()
df_events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63699 entries, 0 to 63698
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       63699 non-null  object
 1   1       13847 non-null  object
 2   2       2614 non-null   object
 3   3       808 non-null    object
 4   4       271 non-null    object
 5   5       73 non-null     object
 6   6       32 non-null     object
 7   7       10 non-null     object
 8   8       4 non-null      object
 9   9       1 non-null      object
 10  10      1 non-null      object
 11  11      1 non-null      object
dtypes: object(12)
memory usage: 5.8+ MB


,0,1,2,3,4,5,6,7,8,9,10,11
0,"{'source': 'PLC', 'message': ' JV-Ti non prêt ...",None,None,None,None,None,None,None,None,None,None,None
1,"{'source': 'iFoil', 'message': ' JV-Ti non prê...","{'source': 'PLC', 'message': ' En attente', 't...",None,None,None,None,None,None,None,None,None,None
2,"{'source': 'PLC', 'message': ' Disponible', 't...",None,None,None,None,None,None,None,None,None,None,None
3,"{'source': 'PLC', 'message': ' Chargeur: mode ...","{'source': 'PLC', 'message': ' Plateau de tête...",None,None,None,None,None,None,None,None,None,None
4,"{'source': 'PLC', 'message': ' Préchauffage', ...",None,None,None,None,None,None,None,None,None,None,None


In [33]:
for col in df_events.columns :
    for i in range(0,len(df_events.iloc[:,col])):
        if isinstance(df_events.loc[i][col], type(None)) :
            df_events.loc[i][col] = np.nan

df_events.info()
df_events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63699 entries, 0 to 63698
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       63699 non-null  object
 1   1       13847 non-null  object
 2   2       2614 non-null   object
 3   3       808 non-null    object
 4   4       271 non-null    object
 5   5       73 non-null     object
 6   6       32 non-null     object
 7   7       10 non-null     object
 8   8       4 non-null      object
 9   9       1 non-null      object
 10  10      1 non-null      object
 11  11      1 non-null      object
dtypes: object(12)
memory usage: 5.8+ MB


,0,1,2,3,4,5,6,7,8,9,10,11
0,"{'source': 'PLC', 'message': ' JV-Ti non prêt ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'source': 'iFoil', 'message': ' JV-Ti non prê...","{'source': 'PLC', 'message': ' En attente', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'source': 'PLC', 'message': ' Disponible', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'source': 'PLC', 'message': ' Chargeur: mode ...","{'source': 'PLC', 'message': ' Plateau de tête...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'source': 'PLC', 'message': ' Préchauffage', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
i = 0
col_names = {}
for col in range(df_events.columns.start, df_events.columns.stop):
    col_names[col] = 'event_'+str(col + 1)
df_events.rename(columns=col_names, inplace=True)

df_events.head()

,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,event_10,event_11,event_12
0,"{'source': 'PLC', 'message': ' JV-Ti non prêt ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'source': 'iFoil', 'message': ' JV-Ti non prê...","{'source': 'PLC', 'message': ' En attente', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'source': 'PLC', 'message': ' Disponible', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'source': 'PLC', 'message': ' Chargeur: mode ...","{'source': 'PLC', 'message': ' Plateau de tête...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'source': 'PLC', 'message': ' Préchauffage', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df = df.drop("events", axis=1)
df.head()

,id,status,created_at
0,4169749,WARNING,2022-04-15 05:55:06.829000
1,4170152,IDLE,2022-04-15 06:06:35.404000
2,4170167,IDLE,2022-04-15 06:07:05.443000
3,4170172,IDLE,2022-04-15 06:07:15.471000
4,4170207,IDLE,2022-04-15 06:08:25.616000


In [36]:
df = df.join(df_events)
df.head()

,id,status,created_at,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,event_10,event_11,event_12
0,4169749,WARNING,2022-04-15 05:55:06.829000,"{'source': 'PLC', 'message': ' JV-Ti non prêt ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4170152,IDLE,2022-04-15 06:06:35.404000,"{'source': 'iFoil', 'message': ' JV-Ti non prê...","{'source': 'PLC', 'message': ' En attente', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4170167,IDLE,2022-04-15 06:07:05.443000,"{'source': 'PLC', 'message': ' Disponible', 't...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4170172,IDLE,2022-04-15 06:07:15.471000,"{'source': 'PLC', 'message': ' Chargeur: mode ...","{'source': 'PLC', 'message': ' Plateau de tête...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4170207,IDLE,2022-04-15 06:08:25.616000,"{'source': 'PLC', 'message': ' Préchauffage', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df.to_csv(path + "metrics_events_split.csv")